In [1]:
# открываю таблички с тест и трейн
import pandas as pd
train_lemm = pd.read_csv('lemm_train_final.csv')
test_lemm = pd.read_csv('lemm_test_final.csv')

In [2]:
# открываю табличку с y_test
test_score = pd.read_csv('test_labels.csv')

In [3]:
# табличка с теми тестовыми комментами, которые не учитываются 
testnotscore = test_score.loc[test_score['toxic'] == -1]

In [4]:
#дропаю из y_test все то, что не учитывается
test_score.drop(test_score[test_score['toxic'] == -1].index, inplace = True)

In [5]:
#список id, которые не учитываются
notscoreid = testnotscore['id'].to_list()

In [6]:
#дропаю все, что не учитывается, из тестового датасета
test_lemm = test_lemm[~test_lemm['id'].isin(notscoreid)]

In [7]:
test_score.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0


In [8]:
#целевые переменные в тесте
toxic_y_test = test_score['toxic'].to_list()
severe_toxic_y_test = test_score['severe_toxic'].to_list()
obscene_y_test = test_score['obscene'].to_list()
threat_y_test = test_score['threat'].to_list()
incult_y_test = test_score['insult'].to_list()
identity_hate_y_test = test_score['identity_hate'].to_list()

In [9]:
# целевые переменные в трейне
toxic_y_train = train_lemm['toxic'].to_list()
severe_toxic_y_train = train_lemm['severe_toxic'].to_list()
obscene_y_train = test_score['obscene'].to_list()
threat_y_train = test_score['threat'].to_list()
incult_y_train = test_score['insult'].to_list()
identity_hate_y_train = test_score['identity_hate'].to_list()

In [10]:
#каунт векторайзер
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
cvec = CountVectorizer(max_features=10000) 
cvec.fit(train_lemm['lemmatized'].values.astype('U'))

X_train_1 = cvec.fit_transform(train_lemm['lemmatized'].values.astype('U'))
X_test_1 = cvec.transform(test_lemm['lemmatized'].values.astype('U'))

In [11]:
#небольшой гридсерч для дерева решений

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'min_samples_leaf': [2, 3, 4, 5, 6], 'max_depth': [2, 3, 4, 5, 6], 'criterion': ['gini', 'entropy']}

est = DecisionTreeClassifier()

gridsearch = GridSearchCV(est, params).fit(X_train_1, toxic_y_train)

In [12]:
print('лучшая модель: ', str(gridsearch.best_params_))
print('точность лучшей модели: ', gridsearch.best_score_)

лучшая модель:  {'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 3}
точность лучшей модели:  0.9370562330332955


In [14]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=6, min_samples_leaf=3).fit(X_train_1, toxic_y_train)

In [15]:
tree.score(X_test_1, toxic_y_test) #точность на лейбле toxic

0.9194254274906999

In [118]:
#тфидф
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
stops = stopwords.words("english")
import numpy as np

text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=200000)
X_train_text = text_transformer.fit_transform(train_lemm['lemmatized'].values.astype('U'))
X_test_text = text_transformer.transform(test_lemm['lemmatized'].values.astype('U'))

In [124]:
#ворд2век
from gensim.models import Word2Vec

sentences = [sentence.split() for sentence in train_lemm['lemmatized'].values.astype('U')]
w2v_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

In [126]:
#ворд2век
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_wv = np.array([vectorize(sentence) for sentence in train_lemm['lemmatized'].values.astype('U')])
X_test_wv = np.array([vectorize(sentence) for sentence in test_lemm['lemmatized'].values.astype('U')])

In [127]:
#логрег
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=16, max_iter = 1000)
x_logreg = logreg.fit(X_train_wv, toxic_y_train)

In [128]:
#предикт
y_pred_toxic = logreg.predict(X_test_wv)

In [129]:
#accuracy
from sklearn import metrics
accuracy_toxic = metrics.accuracy_score( y_pred_toxic, toxic_y_test)
print(accuracy_toxic)

0.9254587514458095
